In [ ]:
%load_ext autoreload
%autoreload 2 
from fibsem import utils, milling, acquire
from fibsem.microscope import FibsemMicroscope
from fibsem.structures import FibsemPatternSettings, FibsemPattern, FibsemStagePosition, BeamType, MicroscopeState, BeamSettings, Point
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
from fibsem import utils
microscope, settings = utils.setup_session()


In [ ]:
# test_state = MicroscopeState(
#     absolute_position=FibsemStagePosition(x=0, y=0, z=0.006, t=0, r=0),
#     eb_settings=BeamSettings(
#         beam_type=BeamType.ELECTRON,
#         working_distance=0.06,
#         beam_current=2.0e-11,
#         voltage=3000,
#         hfw=0.00015,
#         dwell_time=1.0e-06,
#         resolution=[1536, 1024],
#         stigmation=Point(x=0.1, y=0.1),
#         shift=Point(x=1e-6, y=1e-6),
#         scan_rotation=0.0,
#     ),
#     ib_settings=BeamSettings(
#         beam_type=BeamType.ION,
#         working_distance=0.06,
#         beam_current=2.0e-11,
#         voltage=30000,
#         hfw=0.00015,
#         dwell_time=1.0e-06,
#         resolution=[1536, 1024],
#         stigmation=Point(x=0.1, y=0.1),
#         shift=Point(x=1e-6, y=1e-6),
#         scan_rotation=0.0,
#     )
# )

test_state = microscope.get_current_microscope_state()

from pprint import pprint
pprint(test_state.__to_dict__())


In [ ]:
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")
plt.show()

test_state = microscope.get_current_microscope_state()
pprint(test_state.__to_dict__())

test_state.ib_settings.shift = Point(0e-6, 0e-6)
test_state.eb_settings.shift = Point(0e-6, 0e-6)
test_state.ib_settings.stigmation = Point(0e-6, 0e-6)
test_state.eb_settings.stigmation = Point(0e-6, 0e-6)
microscope.set_microscope_state(test_state)

pprint(microscope.get_current_microscope_state().__to_dict__())

eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")
plt.show()

In [ ]:
microscope.set_microscope_state(test_state)

In [ ]:
acquire.take_reference_images(microscope, settings.image)

In [ ]:
return_state = microscope.get_current_microscope_state()
print(return_state)

In [ ]:

rectangle = FibsemPatternSettings(
    pattern = FibsemPattern.Rectangle,
    width = 10e-6,
    height = 10e-6,
    rotation = 0,
    centre_x = 0,
    centre_y = 0,
    depth = 1e-6,
)
#milling.draw_bitmap(microscope, rectangle, "C:\\Users\\lnae0002\\Desktop\\autolamella_test\\2\\000001\\Empty_ref.tif")
settings.image.beam_type = BeamType.ION
img = acquire.new_image(microscope, settings.image)


In [ ]:
milling.milling_protocol(
    microscope,
    settings.milling,
    patterns= [rectangle],
    drift_correction= True,
    image_settings = settings.image,
    ref_image = img,
)